In [117]:
import pandas as pd
import json
import numpy as np

In [ ]:
import os

file_list = []

folder_path = "./rta_player_data"
files_in_folder = os.listdir(folder_path)

# Filter out only files (excluding directories)
files = [file for file in files_in_folder if os.path.isfile(os.path.join(folder_path, file))]

for file in files:
    file_list.append('./rta_player_data/' + file)


In [103]:
file_list

['./rta_player_data/104467101.json',
 './rta_player_data/124552294.json',
 './rta_player_data/133232375.json',
 './rta_player_data/134489859.json',
 './rta_player_data/139771474.json',
 './rta_player_data/155256756.json',
 './rta_player_data/159055138.json',
 './rta_player_data/159620508.json',
 './rta_player_data/163085629.json',
 './rta_player_data/170502668.json',
 './rta_player_data/171276418.json',
 './rta_player_data/177444551.json',
 './rta_player_data/188175150.json',
 './rta_player_data/191603279.json',
 './rta_player_data/191616064.json',
 './rta_player_data/206705999.json',
 './rta_player_data/60090739.json',
 './rta_player_data/60134130.json',
 './rta_player_data/60142509.json',
 './rta_player_data/60318328.json',
 './rta_player_data/61214698.json',
 './rta_player_data/63467428.json',
 './rta_player_data/63908598.json',
 './rta_player_data/66879752.json',
 './rta_player_data/68183898.json',
 './rta_player_data/70726715.json',
 './rta_player_data/79680386.json',
 './rta_play

In [105]:
def drafts(team_1, team_2, deck_1, deck_2, iswin):
    
    data = json.loads(team_1)
    my_team_data = data["my_team"]
    df_team = pd.DataFrame(my_team_data)
    
    # df_team = df_team.drop(['artifact', 'respawn', 'mvp', 'kill_count', 'position', 'grade', 'awaken_grade', 'attribute_cd', 'level', 'job_cd'], axis=1)
    
    data2 = json.loads(team_2)
    my_team_data_enemy = data2["my_team"]
    df_enemy_team = pd.DataFrame(my_team_data_enemy)
    
    if (iswin == 1):
        df_team['is_win'] = iswin
        df_enemy_team['is_win'] = 2
    else:
        df_team['is_win'] = iswin
        df_enemy_team['is_win'] = 1
        
    my_hero_list = json.dumps(deck_1)
    data_dict = json.loads(my_hero_list)
    df_hero_list = pd.DataFrame(data_dict['hero_list'])
    
    enemy_hero_list = json.dumps(deck_2)
    data_dict = json.loads(enemy_hero_list)
    df_enemy_hero_list = pd.DataFrame(data_dict['hero_list'])
    
   
    # df_win = pd.DataFrame(data_win)
    
    # takes both side drafts into one data entry 
    # result = pd.merge(df_team, df_hero_list, on="hero_code")

    
    # result2 = pd.merge(df_enemy_team, df_enemy_hero_list, on="hero_code")

    draft1 = pd.concat([df_enemy_team, df_team], ignore_index=True,axis=0)
    draft2 = pd.concat([df_enemy_hero_list, df_hero_list], ignore_index=True,axis=0)

    draft = pd.concat([draft1, draft2], ignore_index=False,axis=1)
    draft = draft.loc[:,~draft.columns.duplicated()]
    
    
    return draft 

In [91]:
def filter_data(file_name):
    battles = pd.read_json(file_name)
    battles = dict(battles.loc["battle_list"])['result_body']
    battle_data = []

    for x in range(len(battles)):
        battle_data.append(battles[x]['teamBettleInfo'])
        battle_data.append(battles[x]['teamBettleInfoenemy'])
        battle_data.append(battles[x]['my_deck'])
        battle_data.append(battles[x]['enemy_deck'])
        battle_data.append(battles[x]['iswin'])
    
    # stores all generated drafts
    drafts_list = []

    # generate draft data 
    i = 0
    while i < len(battles)*5:
        string_data_team = '{' + battle_data[i] + '}'
        string_data_enemy = '{' + battle_data[i+1] + '}'
        string_data_my_deck = battle_data[i+2] 
        string_data_enemy_deck = battle_data[i+3]
        string_data_win = battle_data[i+4]
        
        res = drafts(string_data_team, string_data_enemy , string_data_my_deck, string_data_enemy_deck, string_data_win)
        drafts_list.append(res)
        i += 5
        
    return drafts_list

In [112]:
drafts_dataset = []
for i in range(len(file_list)):
    res = filter_data(file_list[i])
    drafts_dataset.append(res)

In [121]:
collapsed_arr = [item for sublist in drafts_dataset for item in sublist]

In [133]:
len(collapsed_arr)

3000

In [137]:
concatenated_df = pd.concat(collapsed_arr, ignore_index=True)

# Specify the file name
csv_file = 'drafts_dataset.csv'

# Write the concatenated DataFrame to a CSV file
concatenated_df.to_csv(csv_file, index=True)

